# 說明

0. 接收使用者訊息存到另一張Table
1. 使用者逐步輸入它的限制，但可能會有順序要限制(從年度問到類型)
2. 目前想不到怎麼判斷可以知道使用限制哪一項
3. 照順序
    1. 先問限制年度
    2. 中文片名(可能設個提示說打空白 代表不限制)
    3. 原文片名(可能設個提示說打空白 代表不限制)
    4. 國別(可能設個提示說打空白 代表不限制)
    5. 語言(可能設個提示說打空白 代表不限制)
    6. 類型
4. 輸入完6次後，會得到一個 6 * 7 表格

    | id | 年度 | 中文片名 | 原文片名 | 國別 | 語言 | 類型 |
    | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
    | 1 | 2020< | NULL | NULL | NULL | NULL | NULL |
    | 2 | NULL | 測試! | NULL | NULL | NULL | NULL |
    | 3 | NULL | NULL | TEST! | NULL | NULL | NULL |
    | 4 | NULL | NULL | NULL | 臺灣 | NULL | NULL |
    | 5 | NULL | NULL | NULL | NULL | 中文 | NULL |
    | 6 | NULL | NULL | NULL | NULL | NULL | Comedy Drama |
    
5. 取出需要那個對角線資訊(id 辨識用不會取出)
6. 傳給Recommand_request 去推薦

# 輸入說明
不輸入代表不限制
1. 年份有3種方向
    * 2000> : 2000年之後的
    * 2000< : 2000年之前的
    * 2020-2022 : 有輸入到減號，會去判斷這兩個時間內的
    
2. 中文片名、英文片名、國家、語言都是模糊搜尋，有那個"字"出現就會搜到

    ex : 國家輸入"國", 有這個字就會查, 不管美國、法國、英國還甚麼的

3. 類型可以同時查很多類, 用空白間格(大小寫不拘, Only Eng)

    ex : Action Comedy Thriller, 會找同時符合這3個的


In [13]:
import sqlite3
import random

class Recommand_System():
    def __init__(self):
        self.conn = sqlite3.connect('Movies.db')
        self.cursor = self.conn.cursor()
        self.create_table()

    def create_table(self):
        # 有已存在Table防呆(理想上應該是會一直存在此table，資料只會去覆蓋)
        create_table_query = '''
            CREATE TABLE IF NOT EXISTS `Recommand_request`(
                `id` integer NOT NULL PRIMARY KEY ,
                `年度` VARCHAR(20),
                `中文片名` VARCHAR(100),
                `原文片名` VARCHAR(100),
                `國別` VARCHAR(100),
                `語言` VARCHAR(100),
                `類型` VARCHAR(100)
            )
        '''
        self.cursor.execute(create_table_query)
        self.conn.commit()

    # 插入(replace 固定順序)那些限制
    def insert_data(self, index, **kwargs):
        column_names = ['id', '年度', '中文片名', '原文片名', '國別', '語言', '類型']
        column_values = [index] + [kwargs.get(column, None) for column in column_names[1:]]
        
        placeholders = ', '.join(['?'] * len(column_names))
        column_names_string = ', '.join(column_names)
    
        insert_data_query = f"REPLACE INTO `Recommand_request` ({column_names_string}) VALUES ({placeholders})"
        
        self.cursor.execute(insert_data_query, column_values)
        self.conn.commit()

    def get_diagonal_data(self):        
        select_query = f"SELECT * FROM `Recommand_request`"
        self.cursor.execute(select_query)
        
        req_data = [row[i] for i, row in enumerate(self.cursor.fetchall(), 1)]
        
        return req_data
    
    # 推薦
    def random_recommand(self, lst:list):
        # 匯入使用者限搜
        year = lst[0]
        chinese_title = lst[1]
        original_title = lst[2]
        country = lst[3]
        language = lst[4]
        genre = lst[5]

        query = "SELECT 年度,中文片名,原文片名,國別,語言,類型 FROM Movies_latest WHERE 1=1"
        params = []

        # 年度搜索
        if year:
            if '-' in year:
                # 年份範圍搜索
                start_year, end_year = year.split('-')
                query += " AND 年度 BETWEEN ? AND ?"
                params.extend([start_year, end_year])
            elif year.endswith('>'):
                # 某年之後搜索
                year_offset = year[:-1]
                query += " AND 年度 >= ?"
                params.append(year_offset)
            elif year.endswith('<'):
                # 某年之前搜索
                year_offset = year[:-1]
                query += " AND 年度 <= ?"
                params.append(year_offset)
            else:
                # 指定某年
                query += " AND 年度 = ?"
                params.append(year)

        # 可模糊搜尋
        if chinese_title:
            query += " AND 中文片名 LIKE '%' || ? || '%'"
            params.append(chinese_title)

        # 可模糊搜尋
        if original_title:
            query += " AND 原文片名 LIKE '%' || ? || '%'"
            params.append(original_title)

        # 可模糊搜尋
        if country:
            query += " AND 國別 LIKE '%' || ? || '%'"
            params.append(country)

        # 可模糊搜尋
        if language:
            query += " AND 語言 LIKE '%' || ? || '%'"
            params.append(language)

        # 可多重搜尋
        if genre:
            genre_keywords = genre.split()
            for keyword in genre_keywords:
                query += " AND 類型 LIKE '%' || ? || '%'"
                params.append(keyword)

        
        self.cursor.execute(query, params)
        results = self.cursor.fetchall()
        self.conn.close()

        if not results: return "無結果"

        # 隨機從限搜下推3個
        # 有重複名稱就從新隨機(英文片名判斷)
        if len(results) > 3:
            while True:
                random_results = random.sample(results, 3)
                check = set([item[2] for item in random_results])
                if len(check) == 3: break
        else:
            random_results = results
        
        result = '\n'.join([' | '.join(map(str, tpl)) for tpl in random_results])
        return result

In [16]:
# Test1
RS = Recommand_System()
# 模擬linebot 1來1往6次的情況
# 那個index、欄位名字應該可以先打，剩下是接收使用者傳的訊息
# 現在請輸入你想限制的XXX欄位(輸入空白代表None之類的)

RS.insert_data(1, 年度 = '2020<')
RS.insert_data(2, 中文片名 = '測試!')
RS.insert_data(3, 原文片名 = 'TEST!')
RS.insert_data(4, 國別 = '臺灣')
RS.insert_data(5, 語言 = '中文')
RS.insert_data(6, 類型 = 'Comedy Drama')
result = RS.get_diagonal_data()
print(RS.random_recommand(result))

無結果


In [15]:
# Test2
RS = Recommand_System()
# 模擬linebot 1來1往6次的情況
# 那個index、欄位名字應該可以先打，剩下是接收使用者傳的訊息
# 現在請輸入你想限制的XXX欄位(輸入空白代表None之類的)

RS.insert_data(1, 年度 = '2010>')
RS.insert_data(2, 中文片名 = None)
RS.insert_data(3, 原文片名 = None)
RS.insert_data(4, 國別 = None)
RS.insert_data(5, 語言 = None)
RS.insert_data(6, 類型 = 'Drama')
result = RS.get_diagonal_data()
print(RS.random_recommand(result))

2022 | 即使，這份戀情今晚會從世上消失 | EVEN IF THIS LOVE DISAPPEARS TONIGHT | 日本 | 日語 | Drama, Romance
2013 | unknown | Broken Glass Park | 德國 | 德語 | Drama
2018 | 阿飛正傳 | DAYS OF BEING WILD | 香港 | 粵語 | Crime, Drama, Romance
